In [30]:
import polars as pl

df = pl.read_csv("4DNESN49VY8X_raw_files_2024-08-01-16h-46m.tsv", separator="\t", skip_rows=1).drop_nulls()


In [34]:
import subprocess

subset = df.select(["File Accession", "Related File Relationship", "Related File", "Paired End", "Bio Rep No"]).drop_nulls()
pe1 = subset.filter(pl.col("Paired End") == 1).sort(by="File Accession")
pe2 = subset.filter(pl.col("Paired End") == 2).sort(by="Related File")
for r1, r2 in zip(pe1.iter_rows(), pe2.iter_rows()):
    assert r1[0] == r2[2] and r1[2] == r2[0]
ends = pe1.rename({"File Accession":"End1"}).with_columns(pe2.rename({"File Accession":"End2"})).select(["End1", "End2", "Bio Rep No"]).sort(by="Bio Rep No")
for i, row in enumerate(ends.iter_rows()):
    f1 = f"downsampled/{row[0]}.fastq.gz"
    f2 = f"downsampled/{row[1]}.fastq.gz"
    biorep = row[2]
    f1_new = f"downsampled/4DNESN49VY8X_BR{biorep}_F{i}_R1.fastq.gz"
    f2_new = f"downsampled/4DNESN49VY8X_BR{biorep}_F{i}_R2.fastq.gz"
    cmd = f"mv {f1} {f1_new} && mv {f2} {f2_new}"
    subprocess.run(cmd, shell=True)

In [33]:
!ls

4DNESN49VY8X_raw_files_2024-08-01-16h-46m.tsv  downsampled  Untitled.ipynb
